In [130]:
import numpy as np
import math
import sympy as sp
from IPython.display import Latex, display_latex

np.set_printoptions(suppress=True, linewidth=np.nan)


### Util

In [131]:
def choose_row(col: np.ndarray) -> int:
    for i, r in enumerate(col):
        if r < 0:
            return i

In [132]:
def dual_minimum_ratio_test(row: np.ndarray, c: np.ndarray) -> int:
    c_min = 0
    min_val = math.inf
    for k in range(len(c)):
        if row[k] < 0:
            min_val_temp = c[k] / row[k]
            if min_val_temp < min_val:
                min_val = min_val_temp
                c_min = k
    return c_min if min_val != math.inf else None


In [133]:
def pivoting(tableau: np.ndarray, row: int, col: int):
    # escale pivot row min to 1.0
    tableau[row] = tableau[row] / tableau[row, col]
    # pivot proccess: convert al column to zero except row
    for k in range(len(tableau)):
        if k != row and abs(tableau[k, col]) > 0:
            tableau[k] = tableau[k] - tableau[k, col] * tableau[row, :]


In [134]:
def correct_all_tableau(tableau: np.ndarray, basic_var: list[int]):
    # correct basic variable of all column with value distict of zero
    for index, col in enumerate(basic_var):
        row = index + 1
        pivoting(tableau, row, col)
    print("All tableau corrected")


In [135]:
def real_index(basic_var: list[int]):
    return list(map(lambda x: x + 1, basic_var))


In [136]:
def basic2vertex(tableau: np.ndarray, basic_var: list[int]) -> None:
    # only work with simplex method because m = len(basic_var) and n = len(non_basic_var)
    dim_1, dim_2 = tableau.shape
    m = len(basic_var)
    n = dim_2 - dim_1  # for Big-M don't delete artificial variables
    x = np.zeros(m + n)
    x[basic_var] = tableau[1:, -1]
    return x[:n]


In [137]:
def optimal_solution(tableau: np.ndarray, basic_var: list[int]) -> None:
    print("Optimal solution: x_opt =", basic2vertex(tableau, basic_var))


### Dual Simplex Algorithm

In [141]:
def dual_simplex(tableau: np.ndarray, basic_var: list[int]):
    print(f"vertex = {basic2vertex(tableau, basic_var)}, x_B = {real_index(basic_var)}\n{tableau}")
    while (tableau[1:, -1] < 0).any():
        r_neg = choose_row(tableau[1:, -1]) + 1    # select max negative 
        c_min = dual_minimum_ratio_test(row=tableau[r_neg, :-1], c=tableau[0, :-1])
        if c_min == None:
            print("No feasible solution")
            break
        pivoting(tableau, r_neg, c_min)
        basic_var[r_neg - 1] = c_min
        print(f"vertex = {basic2vertex(tableau, basic_var)}, x_B = {real_index(basic_var)}\n{tableau}")


In [142]:
tableau = np.array([[0, 0, -1.5, -0.5, 0, 0, 0., -22.5],
                    [0, 1, 1.5, -0.5, 0.0, 0, 0, 4.5],
                    [1, 0, -0.5, 0.5, 0.0, 0, 0, 4.5],
                    [0, 0, 0.5, -0.5, 1.0, 0, 0, 2.5],
                    [0, 0, -1.5, 0.5, 0.0, 1, 0, 1.5],
                    [2, 1, 0.0, 0.0, 0.0, 0., 1, 13]])

basic_var = [1, 0, 4, 5, 6]
print(f"x_B = {basic_var}\n{tableau}")
correct_all_tableau(tableau, basic_var)
dual_simplex(tableau, basic_var)
optimal_solution(tableau, basic_var)

x_B = [1, 0, 4, 5, 6]
[[  0.    0.   -1.5  -0.5   0.    0.    0.  -22.5]
 [  0.    1.    1.5  -0.5   0.    0.    0.    4.5]
 [  1.    0.   -0.5   0.5   0.    0.    0.    4.5]
 [  0.    0.    0.5  -0.5   1.    0.    0.    2.5]
 [  0.    0.   -1.5   0.5   0.    1.    0.    1.5]
 [  2.    1.    0.    0.    0.    0.    1.   13. ]]
All tableau corrected
vertex = [4.5 4.5], x_B = [2, 1, 5, 6, 7]
[[  0.    0.   -1.5  -0.5   0.    0.    0.  -22.5]
 [  0.    1.    1.5  -0.5   0.    0.    0.    4.5]
 [  1.    0.   -0.5   0.5   0.    0.    0.    4.5]
 [  0.    0.    0.5  -0.5   1.    0.    0.    2.5]
 [  0.    0.   -1.5   0.5   0.    1.    0.    1.5]
 [  0.    0.   -0.5  -0.5   0.    0.    1.   -0.5]]
vertex = [4. 5.], x_B = [2, 1, 5, 6, 4]
[[  0.   0.  -1.   0.   0.   0.  -1. -22.]
 [  0.   1.   2.   0.   0.   0.  -1.   5.]
 [  1.   0.  -1.   0.   0.   0.   1.   4.]
 [  0.   0.   1.   0.   1.   0.  -1.   3.]
 [  0.   0.  -2.   0.   0.   1.   1.   1.]
 [ -0.  -0.   1.   1.  -0.  -0.  -2.   1.]]
O

In [143]:
tableau = np.array([[-2, -4, -7, 0, 0, 0.],
                    [-2, -1, -6, 1, 0, -5],
                    [-4, 6., -5, 0, 1, -8]])

basic_var = [3, 4]

dual_simplex(tableau, basic_var)
optimal_solution(tableau, basic_var)

vertex = [0. 0. 0.], x_B = [4, 5]
[[-2. -4. -7.  0.  0.  0.]
 [-2. -1. -6.  1.  0. -5.]
 [-4.  6. -5.  0.  1. -8.]]
vertex = [2.5 0.  0. ], x_B = [1, 5]
[[ 0.  -3.  -1.  -1.   0.   5. ]
 [ 1.   0.5  3.  -0.5 -0.   2.5]
 [ 0.   8.   7.  -2.   1.   2. ]]
Optimal solution: x_opt = [2.5 0.  0. ]
